In [7]:
import pandas as pd
import re
import json
import glob
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains

In [8]:
%load_ext jupyternotify

ModuleNotFoundError: No module named 'jupyternotify'

In [ ]:
# Combine CSV, generate output

files = glob.glob("Clean_data/Kickstarter.csv")

df = pd.read_csv("Clean_data/Kickstarter.csv", index_col=None, header=0)

# csv = []

# for file in files:
#     df = pd.read_csv(file, index_col=None, header=0)
#     csv.append(df)

# df = pd.concat(csv, axis=0, ignore_index=True)

In [ ]:
df.head(2)

In [ ]:
print(df.shape, "\n",df.columns)

In [ ]:
# 1. Get main urls for all projects
urls = df.urls

urls_ = list(map(lambda url:json.loads(url)["web"]["project"],urls))
urls = urls_

# 2. Get urls for updates page
updates_urls = []

for i in range(0, len(urls)):
    split_string = urls[i].split("?", 1)
    updates_url = split_string[0] + "/posts"
    updates_urls.append(updates_url)

print(len(updates_urls))

In [9]:
ids = df.id
start_int, end_int = 305,307

In [11]:
from selenium.webdriver.chrome.options import Options

In [12]:
# Run these two lines if chromedriver does not execute (and comment out path and driver lines)
# from webdriver_manager.chrome import ChromeDriverManager
# driver = webdriver.Chrome(ChromeDriverManager().install())

path = "Webdriver/chromedriver.exe"
options = Options()
options.add_argument("--incognito")
driver = webdriver.Chrome(executable_path = path, options=options)

n_updates_all = []
updates_all = []
errors_url = []

for i in tqdm(range(start_int, end_int)):
    driver.get(updates_urls[i])

    # 1. Get total number of updates
    n_update = WebDriverWait(driver,10).until(EC.visibility_of_all_elements_located((By.XPATH,'//*[@id="updates-emoji"]/span')))
    
    for number in n_update:
        n_updates_all.append(number.text)
        
    # 2. Get all updates and save as a list
    updates = []
    try: 
        elements = WebDriverWait(driver,10).until(EC.visibility_of_all_elements_located((By.XPATH,"//article")))
        for element in elements:
            updates.append(element.text)

    except TimeoutException:
        errors_url.append(updates_urls[i])
        pass
    
    updates_all.append(updates)
    time.sleep(1)

driver.quit()

  0%|                                                                                            | 0/2 [00:12<?, ?it/s]


TimeoutException: Message: 


In [ ]:
print('completed for index:', start_int, ' to ', end_int)
print('zeros:', len(errors_url))
print('expected: ', end_int-start_int)
print('success: ', len(n_updates_all), len(updates_all))

In [ ]:
df_export = pd.DataFrame({"id":ids.iloc[start_int:end_int],
                         "update":updates_all,
                         "n_updates":n_updates_all}, index = None)
filename = "Updates_" + str(start_int) + "-" + str(end_int)
df_export.to_csv("Output/Update/{}.csv".format(filename))

In [ ]:
%%notify
print('Scraping completed.')

## Draft Ignore

In [11]:
# from selenium.common.exceptions import TimeoutException
# from selenium.webdriver.common.action_chains import ActionChains

# # Run these two lines if chromedriver does not execute (and comment out path and driver lines)
# # from webdriver_manager.chrome import ChromeDriverManager
# # driver = webdriver.Chrome(ChromeDriverManager().install())

# path = "Webdriver/chromedriver.exe"
# driver = webdriver.Chrome(executable_path = path)


# updates_all = []
# errors_url = []

# for i in tqdm(range(5,10)):
#     driver.get(updates_urls[i])

#     # 1. Load all results
# #     while True:
# #         try:
# #             ActionChains(driver).move_to_element(WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Load more')]")))).pause(5).click().perform()
# #     #         print("LOAD MORE button clicked")
# #         except TimeoutException:
# #     #         print("No more LOAD MORE button to be clicked")
# #             break

#     # 2. Get all updates and save as a list
#     updates = []
#     try: 
#         elements = WebDriverWait(driver,10).until(EC.visibility_of_all_elements_located((By.XPATH,"//article")))
#         for element in elements:
#             updates.append(element.text)

#     except TimeoutException:
#         errors_url.append(urls[i])
    
#     updates_all.append(updates)

# driver.quit()